### 여러 PDF 불러오기

In [6]:
import os
import pymupdf4llm
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

pdf_dir = "E:/work/MS_project_2/data/pdfs"  # 여러 PDF 파일이 들어 있는 디렉토리



### PDF-> 마크다운 형태로 변환
### source 정보 추가

In [7]:
from langchain.schema import Document

all_docs = []

for filename in os.listdir(pdf_dir):
    if filename.endswith(".pdf"):
        pdf_path = os.path.join(pdf_dir, filename)
        md_path = pdf_path.replace(".pdf", ".md")

        # 1. PDF → Markdown 저장
        md_text = pymupdf4llm.to_markdown(pdf_path)
        with open(md_path, "w", encoding="utf-8") as f:
            f.write(md_text)

        # 2. Markdown 로딩
        loader = TextLoader(md_path, encoding="utf-8")
        documents = loader.load()

        # 3. ✅ source 메타데이터 추가
        for doc in documents:
            doc.metadata["source"] = filename

        # 4. ✅ 분할 (source 정보가 포함된 문서 기준으로)
        docs_split = text_splitter.split_documents(documents)

        # 5. ✅ all_docs에 청크된 문서 저장
        all_docs.extend(docs_split)


Processing E:/work/MS_project_2/data/pdfs\서울지역본부 청년 매입임대주택.pdf...
[                                        ] (0/2[=                                       ] ( 1/2=[===                                     ] ( 2/2[====                                    ] ( 3/25=[======                                  ] ( 4/25=[========                                ] ( 5/25[=========                               ] ( 6/2=[===========                             ] ( 7/2[============                            ] ( 8/25=[==============                          ] ( 9/25=[================                        ] (10/25[=================                       ] (11/2=[===================                     ] (12/2[====================                    ] (13/25=[======================                  ] (14/25=[========================                ] (15/25[=========================               ] (16/2=[===========================             ] (17/2[============================            ] (18/25=[=

In [11]:
print(len(docs_split))
print(len(all_docs))

44
169


### 임베딩 객체 생성

In [ ]:
from langchain_openai import AzureOpenAIEmbeddings
import os

embedding_api_key = ""
embedding_endpoint = ""
embedding_api_version = "2024-02-15-preview"
embedding_deployment = "text-embedding-3-small"

os.environ.pop("OPENAI_API_BASE", None)
os.environ.pop("BASE_URL", None)
embedding = AzureOpenAIEmbeddings(
    api_key=embedding_api_key,
    azure_endpoint=embedding_endpoint,
    model=embedding_deployment,  # 배포 이름 확인 필수
    openai_api_version=embedding_api_version
)


### AI search 연동

### 인덱스 먼저 생성

In [ ]:
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SearchIndex,
    SimpleField,
    SearchFieldDataType,
    VectorSearch,
    HnswAlgorithmConfiguration,
    VectorSearchAlgorithmKind,
    SearchField,
    VectorSearchProfile
)

# 설정
ai_search_endpoint = ""
ai_search_api_key = ""
ai_search_index_name = "pdf_all_index"
embedding_dim = 1536

# 클라이언트 생성
index_client = SearchIndexClient(
    endpoint=ai_search_endpoint,
    credential=AzureKeyCredential(ai_search_api_key)
)

# ✅ 벡터 검색 프로파일 정의 (신규!)
vector_search = VectorSearch(
    profiles=[
        VectorSearchProfile(
            name="default",
            algorithm_configuration_name="my-algorithm"  # 🔥 정확한 인자 이름
        )
    ],
    algorithms=[
        HnswAlgorithmConfiguration(
            name="my-algorithm",
            kind=VectorSearchAlgorithmKind.HNSW
        )
    ]
)

# ✅ 인덱스 필드 정의 (vectorSearchProfile 사용)
fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True),
    SearchField(name="content", type=SearchFieldDataType.String, searchable=True),
    SearchField(name="source", type=SearchFieldDataType.String, searchable=True, filterable=True),
    SearchField(
        name="embedding",
        type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
        searchable=True,
        vector_search_dimensions=1536,
        vector_search_profile_name="default"  # 기존의 'vector_search_configuration' 대신 사용
    )
]

# 인덱스 객체 생성
index = SearchIndex(
    name=ai_search_index_name,
    fields=fields,
    vector_search=vector_search
)

# 존재 시 삭제 후 재생성
if ai_search_index_name in [i.name for i in index_client.list_indexes()]:
    index_client.delete_index(ai_search_index_name)

index_client.create_index(index)
print("✅ Azure Search 인덱스 생성 완료 (최신 버전 호환)")


✅ Azure Search 인덱스 생성 완료 (최신 버전 호환)


In [17]:
#print(all_docs[0].metadata)
print(all_docs[0])
print(all_docs[0].page_content)
print(all_docs[0].metadata)

page_content='###### 송파구, 용산구, 은평구, 종로구, 중구]


-----

###### ■ 공급대상 주택 : 총 190호

 • 주택군, 주택별 소재지, 면적, 임대조건 등 세부내역은 첨부 “주택내역” 참조

  * 위 공급호수는 기존에 선정된 예비입주자 계약 및 신규 매입물량 추가 등으로 변경될 수 있습니다.

  * 청약플러스(https://apply.lh.or.kr) 공고문 페이지에 주택사진이 게시되어 있으므로 참고하시기 바랍니다.

  * 아동복지시설(가정위탁 포함) 및 청소년쉼터 퇴소자 수시 우선공급으로 인해 지역별 공급물량이 

 감소될 수 있습니다.

  * 본 모집공고의 예비입주자로 선정되었다 하더라도 기존 임차인의 퇴거 및 임대공급 시행 전 실시하는

 주택 개보수 완료 상황 등에 따라 주택물량이 감소하거나 입주까지 상당기간이 소요될 수 있습니다.

 ■ 임대기간 및 임대조건

 구분 내용' metadata={'source': '서울지역본부 청년 매입임대주택.pdf'}
###### 송파구, 용산구, 은평구, 종로구, 중구]


-----

###### ■ 공급대상 주택 : 총 190호

 • 주택군, 주택별 소재지, 면적, 임대조건 등 세부내역은 첨부 “주택내역” 참조

  * 위 공급호수는 기존에 선정된 예비입주자 계약 및 신규 매입물량 추가 등으로 변경될 수 있습니다.

  * 청약플러스(https://apply.lh.or.kr) 공고문 페이지에 주택사진이 게시되어 있으므로 참고하시기 바랍니다.

  * 아동복지시설(가정위탁 포함) 및 청소년쉼터 퇴소자 수시 우선공급으로 인해 지역별 공급물량이 

 감소될 수 있습니다.

  * 본 모집공고의 예비입주자로 선정되었다 하더라도 기존 임차인의 퇴거 및 임대공급 시행 전 실시하는

 주택 개보수 완료 상황 등에 따라 주택물량이 감소하거나 입주까지 상당기간이 소요될 수 있습니다.

 ■ 임대기간 및 임대조건

 구분 내용
{'source': '서울지역본부 청년 매입임대주택.pd

In [ ]:
from langchain_community.vectorstores import AzureSearch
ai_search_api_key = ''
ai_search_index_name = 'pdf_all_index'
ai_search_service_name="6b034-ai-search"
ai_search_endpoint = ''


vectorstore = AzureSearch(
    azure_search_endpoint=ai_search_endpoint,
    azure_search_key=ai_search_api_key,
    index_name=ai_search_index_name,
    embedding_function=embedding.embed_query,
    metadata_field="source" ## 개중요중요
)

### 데이터 업로드

In [ ]:
# ✅ 1. 문서 내용과 메타데이터 분리
texts = [doc.page_content for doc in all_docs]
metadatas = [{"source": doc.metadata.get("source", "")} for doc in all_docs]



In [26]:
#print(texts[0])
print(metadatas[0])

{'source': '서울지역본부 청년 매입임대주택.pdf'}


In [27]:
from pprint import pprint
pprint(metadatas[:3])

[{'source': '서울지역본부 청년 매입임대주택.pdf'},
 {'source': '서울지역본부 청년 매입임대주택.pdf'},
 {'source': '서울지역본부 청년 매입임대주택.pdf'}]


In [25]:
# ✅ 2. 업로드
vectorstore.add_texts(texts=texts, metadatas=metadatas)


HttpResponseError: () The request is invalid. Details: The property 'metadata' does not exist on type 'search.documentFields' or is not present in the API version '2024-07-01'. Make sure to only use property names that are defined by the type.
Code: 
Message: The request is invalid. Details: The property 'metadata' does not exist on type 'search.documentFields' or is not present in the API version '2024-07-01'. Make sure to only use property names that are defined by the type.

### 확인

In [ ]:
filter_condition = "source eq '서울지역본부 청년 매입임대주택.pdf'"

results = vectorstore.similarity_search_with_score(
    query="신청 자격 조건은?",
    k=3,
    filters=filter_condition  
)

for i, (doc, score) in enumerate(results):
    print(f"[출처: {doc.metadata['source']} | score: {score:.3f}]\n{doc.page_content}")


HttpResponseError: () Invalid expression: Could not find a property named 'source' on type 'search.document'.
Parameter name: $filter
Code: 
Message: Invalid expression: Could not find a property named 'source' on type 'search.document'.
Parameter name: $filter